In [19]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *


In [4]:
# Parameters

ntypes = 2
nperiods = 5
maxstart0 = 5

generators = [1, 1]
period_hours = [6, 3, 6, 3, 6]
demand = [400]
min_load = [850, 1250]
max_load = [2000, 1750]
base_cost = [1000, 2600]
per_mw_cost = [2, 1.3, ]
startup_cost = [2000, 1000]

In [5]:
model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-07


In [6]:
# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

In [7]:
# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

In [8]:
# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))

In [9]:
# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))

In [10]:
# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

In [11]:
# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj)

In [18]:
#model.write('C:/Users/uer/OneDrive - Universidad de los andes/2022-2/tesis final/modelo gurobi/junk.csv')
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0xd010fb19
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolved: 54 rows, 44 columns, 130 nonzeros

Continuing optimization...


Cutting planes:
  MIR: 4
  StrongCG: 1

Explored 1 nodes (23 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 7: 1.00254e+06 1.0057e+06 1.00597e+06 ... 1.12391e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 1.002540000000e+06, best bound 1.002540000000e+06, gap 0.0000%
